In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt, ticker
import matplotlib as mpl
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1"

def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

In [ ]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74' #{R_A}, {Z_A}
basis = 'sto-3g' # {\phi_\mu}
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)

In [ ]:
# obtain the matrices that are defined as soon as the molecule is defined
s1e = h2_0_1.intor_symmetric('int1e_ovlp') # S_\mu\nv
t = h2_0_1.intor_symmetric('int1e_kin')
en = h2_0_1.intor_symmetric('int1e_nuc')
h1e = t + en # H^core_\mu\nv
eri = h2_0_1.intor_symmetric('int2e') # (\mu\nu|\lambda\sigma)

In [ ]:
# obtain F12s at different mo coefficients
F12s = []
c2s = []
c1s = np.linspace(0,0.55,56)
for c1 in c1s:
    c2 = (1+(c1**2)*(s1e[0][1]**2-1)) ** 0.5 - c1*s1e[0][1]
    c2s.append(c2)
    mo_coeff = np.array([[c1, c1], [c2, -c2]])
    mo_occ = np.array([2, 0])
    dms = make_rdm1(mo_coeff, mo_occ) # calculate P
    # initiate mean field calculations
    h2_0_1_rhf = scf.RHF(h2_0_1)
    # obtain the needed matrices
    vhf = h2_0_1_rhf.get_veff(h2_0_1, dms)
    H = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, h1e, mo_coeff]) # calculate H^core
    V = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, vhf, mo_coeff]) # calculate G
    F = H+V # calculate F
    F12 = F[0][1]
    F12s.append(F12)

In [ ]:
# plot the change of F12 vs mo coefficient
xlabel = 'c1**2+c1*c2*S12'
ylabel = 'F12 / Eh'
title = 'Off-diagonal of Fock matrix vs mo coefficient'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
fig, ax = plt.subplots(figsize=(7,6))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
ax.set_ylim(0,0.4)
ax.set_xlim(0,0.6)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
ax.scatter(c1s**2+c1s*c2s*s1e[0][1], F12s, color='k')
fig.tight_layout()